---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [275]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [382]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [383]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    available_states = states.values() 
    with open('university_towns.txt', 'r') as f_in:
        data = f_in.readlines()
        data = [re.sub(r'\s*[\[\()].*\n|\n', '', d)  for d in data] # regex which removes starting from ' [', ' (' or '\n'
        
        _ = [d if d in available_states else None for d in data] # create a list of state names and Nones 
        university_states = pd.Series(_).fillna(method = 'ffill').replace() # forward fill Nones with the preceding state names
        universities = pd.Series(data)
        df = pd.DataFrame({'StateName': university_states, 'RegionName': universities}, columns = ['StateName', 'RegionName'])
        df.drop((df.where(df['RegionName'].isin(available_states))
             .dropna()
             .index), inplace = True, axis = 0) # remove state names from universities column  
    
    return df

In [384]:
get_list_of_university_towns().head(1)

,StateName,RegionName
1,Alabama,Auburn


In [385]:
df = pd.read_excel('gdplev.xls', skiprows = 7)

In [386]:
(df.drop(labels = 
         ['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3', 'Unnamed: 7'], 
         axis = 1,
         inplace = True))

In [387]:
(df.rename(columns = {'Unnamed: 4': 'Quarter', 'Unnamed: 5':'GDP', 'Unnamed: 6':'GDP_adj'}, 
           inplace = True))

In [388]:
df = (df.where(df['Quarter'] >= '2000')
 .dropna())

In [389]:
df.set_index('Quarter', inplace = True)

In [390]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    _ = (df['GDP_adj'] >= df['GDP_adj'].shift()).rolling(window = 2).mean()
    recession_start = _.where(_.shift(-1) == 0).dropna().head(1)
    recession_start.index.get_level_values(level=0).values[0]
    
    return recession_start.index.get_level_values(level=0).values[0]

In [391]:
recession_start = get_recession_start()
print(recession_start)

2008q3


In [392]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    _ = (df['GDP_adj'] > df['GDP_adj'].shift()).rolling(window = 2).mean()
    
    recession_end = (_.where(
        (_.shift() == 1) & (_.index.get_level_values(level=0) > recession_start)
        )
        .dropna())
    
    return recession_end.index.get_level_values(level=0).values[0]

In [393]:
recession_end = get_recession_end()
print(recession_end)

2010q1


In [394]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    _ = df.loc[(df.index.get_level_values(level=0) >= recession_start) & (df.index.get_level_values(level=0) <= recession_end)]
    
    return re.search(r'\d{4}.*', _['GDP_adj'].argmin()).group()

In [395]:
recession_bottom  = get_recession_bottom()
print(recession_bottom)

2009q2


In [396]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df_housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    #dropping irrelevant columns
    df_housing.drop(labels = df_housing.iloc[:, np.r_[0,3:51]].columns, axis = 1, inplace = True) 
    #storing year-month column names in a temporary variable to convert into quarters
    _ = df_housing.iloc[:, 2:]
    #convert column names to quarters (2000-01,2000-02,2000-03 -> 2000Q1,2000Q1,2000Q1 etc.)
    _.columns = pd.PeriodIndex(df_housing.columns[2:], freq='Q').to_series().astype(str)
    #groupby quarters and calculate mean
    _ = _.groupby(_.columns, axis = 1).mean()
    #merge State and RegionName with above quarter data
    df_housing = df_housing.iloc[:,:2].merge(_, left_index = True, right_index = True) 
    df_housing.set_index(['State', 'RegionName'], inplace = True)
    return df_housing

In [397]:
convert_housing_data_to_quarters().head(1)

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.0,528066.666667,532266.666667,540800.0,557200.0,572833.333333,582866.666667,591633.333333,587200.0


In [398]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    _ = convert_housing_data_to_quarters()
    quarter_before_recession = _.columns[_.columns.get_loc(recession_start.upper()) - 1]
    bottom_quarter = _.columns[_.columns.get_loc(recession_bottom.upper())]
    
    univ_town = get_list_of_university_towns()
    univ_town['univ_town'] = True
    
    df = pd.merge(_.reset_index(), univ_town, how='outer')
    df['univ_town'] = df['univ_town'].replace({np.NaN: False})
    
    df.dropna(subset=[quarter_before_recession, bottom_quarter], inplace = True)
    
    df['price_ratio'] = df.loc[:, quarter_before_recession]/df.loc[:, bottom_quarter]
    
    univ_town_price_ratio = df[df['univ_town'] == True]['price_ratio']
    non_univ_town_price_ratio = df[df['univ_town'] == False]['price_ratio']
    
    better = ''
    
    if univ_town_price_ratio.mean() > non_univ_town_price_ratio.mean():
        better = 'university town'
    else:
        better = 'non-university town'
        
    st, p = ttest_ind(univ_town_price_ratio, non_univ_town_price_ratio, nan_policy='omit')
    
    return (True, p, better)

In [399]:
run_ttest()

(True, 9.5977942488634693e-07, 'non-university town')